In [1]:
#LIBRERÍAS

import pandas as pd
import numpy as np
import keras_metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix


file= 'infarto.csv'
dt=pd.read_csv(file, sep=";")


#Aunque no seamos expertos en el tema entendemos que las columnas casado y trabajo no aportan nada para el modelo
#La residencia si porque la calidad del aire es diferente

dt=dt.drop(['id','casado', 'trabajo'], axis=1)

print("Numero de filas con NaN values:\n",dt.isnull().sum())
print("\n\nComo no hay muchos NaN las filas que cuenten con valores nulos las descartamos\n\n\n")
#dt=dt.drop(dt.index[1000:len(dt)])
dt=dt.dropna()
dt.head(5)

Numero de filas con NaN values:
 genero                    0
edad                      0
hipertensión              0
enfermedad_coronaria      0
residencia                0
nivel_glucosa             0
bmi                     201
uso_tabaco                0
infarto                   0
dtype: int64


Como no hay muchos NaN las filas que cuenten con valores nulos las descartamos





,genero,edad,hipertensión,enfermedad_coronaria,residencia,nivel_glucosa,bmi,uso_tabaco,infarto
0,Masculino,67.0,0,1,Urbana,228.69,36.6,Ex fumador,1
2,Masculino,80.0,0,1,Rural,105.92,32.5,Nunca,1
3,Femenino,49.0,0,0,Urbana,171.23,34.4,Fumador,1
4,Femenino,79.0,1,0,Rural,174.12,24.0,Nunca,1
5,Masculino,81.0,0,0,Urbana,186.21,29.0,Ex fumador,1


In [2]:

#Transformamos las columnas necesarias con to_categorical

#sacamos las columnas
columna1=np.array(dt["genero"])
columna2=np.array(dt["residencia"])
columna3=np.array(dt["uso_tabaco"])
label_encoder = LabelEncoder()

#LAS PASAMOS A 0s y 1s
genero = label_encoder.fit_transform(columna1)
residencia= label_encoder.fit_transform(columna2)
tabaco= label_encoder.fit_transform(columna3)

#LAS DEVOLVEMOS AL DATASET
dt["genero"]= k.to_categorical(genero)
dt["residencia"]= k.to_categorical(residencia)
dt["uso_tabaco"]= k.to_categorical(tabaco)

In [3]:
#NORMALIZAMOS LOS DATOS DEL DATASET
def minmax_norm(datos_input): 
    return (datos_input - dt.min()) / ( dt.max() - dt.min())
dt=minmax_norm(dt)
dt.head(5)

#OBTENEMOS LA COLUMNA CON LOS RESULTADOS ESPERADOS Y EL RESTO DE LOS DATOS
esperada= dt.iloc[:, 8]
datos= dt.iloc[:,0:8]


In [4]:


#ENTRENAMIENTO, TEST, PREDICCION

#NO INFARTOS
x_train, x_resto, y_train, y_resto= train_test_split(datos[209::3], esperada[209::3], test_size=0.3)
x_test, x_pred, y_test, y_pred = train_test_split(x_resto, y_resto, test_size=0.2)

#INFARTOS
x_in_train, x_in_resto,y_in_train,y_in_resto = train_test_split(datos[:209], esperada[:209], test_size=0.2)
x_in_test, x_in_pred,y_in_test,y_in_pred = train_test_split(x_in_resto, y_in_resto, test_size=0.2)


#Sumamos los casos para que la red cuente con ambos casos en cada etapa
x_train=pd.concat([x_in_train, x_train], axis=0)
x_test=pd.concat([x_in_test, x_test], axis=0)
x_pred=pd.concat([x_in_pred, x_pred], axis=0)

y_train=pd.concat([y_in_train, y_train], axis=0)
y_test=pd.concat([y_in_test, y_test], axis=0)
y_pred=pd.concat([y_in_pred, y_pred], axis=0)



In [ ]:
#PROCEDEMOS A CREAR LA RED

model=Sequential()

#APLICANDO LA FORMULA SABEMOS QUE EN LA CAPA OCULTA DEBEMOS TENER ENTRE 245 Y 981 NEURONAS
model.add(Dense(500, 'sigmoid', 8))
model.add(Dense(400, 'relu'))
model.add(Dense(1, 'sigmoid'))# UNA SALIDA (0/1)

model.compile(loss="mean_squared_error", optimizer='adam',metrics="accuracy")
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100) #PARA EL CALLBACK Y EVITAR EL SOBREENTRENAMIENTO
model.fit(x=x_train,y=y_train,validation_split=0.2, epochs=1000, shuffle=True, use_multiprocessing=True) #SHUFFLE EN TRUE PARA ENTRENAR CON DATOS LO MAS AL AZAR POSIBLE
#steps_per_epoch=50
#, callbacks=es

Epoch 1/1000
32/32 [==============================] - 1s 30ms/step - loss: 0.1759 - accuracy: 0.7663 - val_loss: 1.4186e-10 - val_accuracy: 1.0000
Epoch 2/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.1696 - accuracy: 0.8304 - val_loss: 7.4472e-11 - val_accuracy: 1.0000
Epoch 3/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.1757 - accuracy: 0.8243 - val_loss: 7.5634e-11 - val_accuracy: 1.0000
Epoch 4/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.1774 - accuracy: 0.8226 - val_loss: 8.0018e-11 - val_accuracy: 1.0000
Epoch 5/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.1633 - accuracy: 0.8367 - val_loss: 8.5640e-11 - val_accuracy: 1.0000
Epoch 6/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.1561 - accuracy: 0.8439 - val_loss: 9.2681e-11 - val_accuracy: 1.0000
Epoch 7/1000
32/32 [==============================] - 0s 3ms/step - loss: 0.1584 - accuracy: 0.8416 - val_loss: 1.0157e-10 

In [ ]:

#UTILIZAMOS LOS DATOS QUE HABIAMOS RESERVADO PARA VER QUE NOS DEVUELVE EL MODELO
y_pred1 = model.predict(x_pred) 
y_pred1 = y_pred1.round(0)
#y_pred1 = np.argmax(y_pred1, axis=1)
scores = model.evaluate(x_test, y_test)#EVALUACION
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#y_pred1

In [ ]:
#OBTENEMOS LAS MÉTRICAS DE PRECISION Y RECALL

print("Precision: ",precision_score(y_pred, y_pred1 , average="macro"))
print("Recall: ",recall_score(y_pred, y_pred1 , average="macro"))

In [ ]:
#obtenemos la matriz de confusion y la mostramos
cm = confusion_matrix(y_pred, y_pred1)
print(cm)
